## ------------------------ D A T A --- E X P L O R A T I O N ------------------------

### We start by importing the necessary libraries

In [42]:
import pandas as pd
import numpy as np
import chart_studio.plotly as py
import cufflinks as cf
import seaborn as sns
import plotly.express as px
import requests
import re
import json

%matplotlib inline

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from datetime import datetime

init_notebook_mode(connected=True)
cf.go_offline()

In [2]:
pd.options.display.max_rows = 999

In [3]:
df = pd.read_csv('datasets/AccidentesAviones.csv')
df.head()

,Unnamed: 0,fecha,HORA declarada,Ruta,OperadOR,flight_no,route,ac_type,registration,cn_ln,all_aboard,PASAJEROS A BORDO,crew_aboard,cantidad de fallecidos,passenger_fatalities,crew_fatalities,ground,summary
0,0,"September 17, 1908",1718,"Fort Myer, Virginia",Military - U.S. Army,?,Demonstration,Wright Flyer III,?,1,2,1,1,1,1,0,0,"During a demonstration flight, a U.S. Army fly..."
1,1,"September 07, 1909",?,"Juvisy-sur-Orge, France",?,?,Air show,Wright Byplane,SC1,?,1,0,1,1,0,0,0,Eugene Lefebvre was the first pilot to ever be...
2,2,"July 12, 1912",0630,"Atlantic City, New Jersey",Military - U.S. Navy,?,Test flight,Dirigible,?,?,5,0,5,5,0,5,0,First U.S. dirigible Akron exploded just offsh...
3,3,"August 06, 1913",?,"Victoria, British Columbia, Canada",Private,?,?,Curtiss seaplane,?,?,1,0,1,1,0,1,0,The first fatal airplane accident in Canada oc...
4,4,"September 09, 1913",1830,Over the North Sea,Military - German Navy,?,?,Zeppelin L-1 (airship),?,?,20,?,?,14,?,?,0,The airship flew into a thunderstorm and encou...


In [4]:
df.shape, df.columns

((5008, 18),
 Index(['Unnamed: 0', 'fecha', 'HORA declarada', 'Ruta', 'OperadOR',
        'flight_no', 'route', 'ac_type', 'registration', 'cn_ln', 'all_aboard',
        'PASAJEROS A BORDO', 'crew_aboard', 'cantidad de fallecidos',
        'passenger_fatalities', 'crew_fatalities', 'ground', 'summary'],
       dtype='object'))

In [5]:
df.drop('Unnamed: 0', axis=1).duplicated().value_counts()

False    5008
dtype: int64

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5008 entries, 0 to 5007
Data columns (total 18 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Unnamed: 0              5008 non-null   int64 
 1   fecha                   5008 non-null   object
 2   HORA declarada          5008 non-null   object
 3   Ruta                    5008 non-null   object
 4   OperadOR                5008 non-null   object
 5   flight_no               5008 non-null   object
 6   route                   5008 non-null   object
 7   ac_type                 5008 non-null   object
 8   registration            5008 non-null   object
 9   cn_ln                   5008 non-null   object
 10  all_aboard              5008 non-null   object
 11  PASAJEROS A BORDO       5008 non-null   object
 12  crew_aboard             5008 non-null   object
 13  cantidad de fallecidos  5008 non-null   object
 14  passenger_fatalities    5008 non-null   object
 15  crew

In [7]:
for col in df:
    print(f'----------{col}----------\n{df[col].value_counts()}\n\n')

----------Unnamed: 0----------
0       1
3336    1
3343    1
3342    1
3341    1
       ..
1668    1
1667    1
1666    1
1665    1
5007    1
Name: Unnamed: 0, Length: 5008, dtype: int64


----------fecha----------
August 31, 1988       4
September 11, 2001    4
February 23, 1945     3
December 28, 1946     3
August 27, 1992       3
                     ..
December 04, 1958     1
December 23, 1958     1
December 24, 1958     1
December 26, 1958     1
July 06, 2021         1
Name: fecha, Length: 4577, dtype: int64


----------HORA declarada----------
?       1504
1500      35
1400      30
1100      30
1700      29
        ... 
2129       1
1002       1
1151       1
0909       1
1914       1
Name: HORA declarada, Length: 1217, dtype: int64


----------Ruta----------
Moscow, Russia                                         16
Manila, Philippines                                    15
New York, New York                                     14
Sao Paulo, Brazil                                   

Now we will delete the 'Unnamed: 0' column, as it contains a unique identifier for each row and is not needed. We will also rename the others columns in order to make more clear what information is contained on each of them.

Both of this changes will be performed in a copy of the original dataset so we can preserve the original data and access it every time we need to.

In [8]:
df2 = df.copy()

In [9]:
df.columns

Index(['Unnamed: 0', 'fecha', 'HORA declarada', 'Ruta', 'OperadOR',
       'flight_no', 'route', 'ac_type', 'registration', 'cn_ln', 'all_aboard',
       'PASAJEROS A BORDO', 'crew_aboard', 'cantidad de fallecidos',
       'passenger_fatalities', 'crew_fatalities', 'ground', 'summary'],
      dtype='object')

In [10]:
new_col_names = {'fecha':'date',
                 'HORA declarada': 'time',
                 'Ruta': 'accident_location',
                 'OperadOR': 'operator',
                 'PASAJEROS A BORDO': 'passengers_aboard',
                 'crew_aboard': 'crew_aboard',
                 'cantidad de fallecidos': 'total_fatalities',
                 'ground': 'ground_fatalities'}

df2.rename(columns = new_col_names, inplace = True)
df2.columns

Index(['Unnamed: 0', 'date', 'time', 'accident_location', 'operator',
       'flight_no', 'route', 'ac_type', 'registration', 'cn_ln', 'all_aboard',
       'passengers_aboard', 'crew_aboard', 'total_fatalities',
       'passenger_fatalities', 'crew_fatalities', 'ground_fatalities',
       'summary'],
      dtype='object')

In [11]:
df2.drop('Unnamed: 0', axis=1, inplace=True)
df2.columns

Index(['date', 'time', 'accident_location', 'operator', 'flight_no', 'route',
       'ac_type', 'registration', 'cn_ln', 'all_aboard', 'passengers_aboard',
       'crew_aboard', 'total_fatalities', 'passenger_fatalities',
       'crew_fatalities', 'ground_fatalities', 'summary'],
      dtype='object')

## Per column data transformations

### • Date and time columns

We will start by transforming the strings on this column into datetime objects and then creating a new column with only the year from it.

In [12]:
format_data = '%B %d, %Y'
def str_2_dt(val):
    return datetime.strptime(val, format_data)
def dt_year(val):
    return val.year

In [13]:
df2['date'] = df2['date'].apply(str_2_dt)
df2['year'] = df2['date'].apply(dt_year)
df2.year.value_counts().head(10)

1946    88
1989    83
1947    82
1948    78
1962    78
1972    77
1945    75
1951    75
1994    74
1970    73
Name: year, dtype: int64

In [14]:
df2.year.min(), df2.year.max()

(1908, 2021)

In [15]:
df2.loc[df2.year == 2021].shape

(7, 18)

In [16]:
for x in df2.time.unique():
    print(x)

1718
?
0630
1830
1030
0100
1520
2345
0515
0845
0700
0745
2130
1000
1655
1215
1740
1140
1315
1645
1342
1730
c 18:00
c 02:30
c 12:00
0530
0930
1527
c 13:15
0800
0830
0405
10:00
14:48
c 11:00
c 9:15
11:30
c 10:00
08:00
10:45
1700
09:18
1145
1130
2100
1101
1800
1010
0935
c 17:00
18:23
17:30
c:17:00
15:00
c 15:45
02:05
09:45
c: 2:00
2240
1300
1045
2330
2230
0900
0840
2000
0400
0648
0545
c 16:00
02:00
1930
1200
c 08:00
0010
c:09:00
0600
02:07
2007
14:30
1230
0740
1500
0315
1540
1120
2235
1600
c 13:00
1815
1415
0500
1100
0950
2245
1050
2342
0730
0333
0252
1900
0045
1518
0330
1835
2225
1436
0940
1420
2018
2008
2300
0955
0219
1940
0440
1915
1932
0352
1910
1020
0520
2200
0300
1400
0338
1047
0314
0323
2047
1938
1107
2050
1340
1840
1925
1040
1225
0423
1650
1530
1507
0803
c 20:40
1430
0445
21:30
1407
2217
0256
0411
c 23:00
1345
10:50
1841
0051
0755
c 09:00
1630
2115
0759
0945
1317
1448
0925
1620
1850
1635
1410
0815
1320
1015
1441
0442
1502
1748
1205
0413
0200
1150
1550
0915
c 20:35
0204
2210
0555
1

In [17]:
time_dtypes = set()
for x in df2.time:
    time_dtypes.add(type(x))
print(time_dtypes)

{<class 'str'>}


In [18]:
time_1st_char = {}
for x in df2.time:
    if x[0] in time_1st_char.keys():
        time_1st_char[x[0]] += 1
    else:
        time_1st_char[x[0]] = 1
print(time_1st_char)

{'1': 1884, '?': 1504, '0': 1078, '2': 503, 'c': 38, '9': 1}


In [19]:
for x in df2.time:
    if x[0] == '9':
        print(x)


900


In [20]:
for x in df2.time:
    if x[0] == 'c':
        print(x)

c 18:00
c 02:30
c 12:00
c 13:15
c 11:00
c 9:15
c 10:00
c 17:00
c:17:00
c 15:45
c: 2:00
c 16:00
c 08:00
c:09:00
c 13:00
c 20:40
c 23:00
c 09:00
c 10:00
c 20:35
c 20:40
c 15:00
c 11:15
c 01:00
c 02:00
c 03:30
c 20:15
c 10:30
c 20:00
c 13:40
c 11:30
c 02:50
c 11:45
c 20:15
c 06:30
c 02:00
c 12:00
c 07:00


In [21]:
for x in df2.time:
    if x[-1] == 'Z':
        print(x)

02:09Z
03:50Z
17:34Z
01:00Z
0500Z


In [22]:
'0000' in df2.time.unique(), '00:00' in df2.time.unique()

(True, False)

In [23]:
midnight_count = 0
for x in df2.time:
    if x == '0000':
        midnight_count +=1
print(midnight_count)

1


In [24]:
def time_formatter(series):
    new_series = []
    for i in range(len(series)):
        if series[i][0:2] == 'c ':
            time_ok = series[i].replace('c ', '')
        elif series[i][0:3] == 'c: ':
            time_ok = series[i].replace('c: ', '')
        elif series[i][0:2] == 'c:':
            time_ok = series[i].replace('c:', '')
        elif series[i][-1] == 'Z':
            time_ok = series[i].replace('Z', '')
        elif series[i][0] == '?':
            time_ok = '00:00'
        else:
            time_ok = series[i]
        if len(time_ok) < 4:
            time_ok = time_ok.zfill(4)
        if (len(time_ok) == 4):
            cnt = 0
            for x in range(4):
                if time_ok[x] == ':':
                    cnt += 1
            if cnt > 0:
                spl = time_ok.split(':')
                if len(spl[0]) == 1:
                    spl[0] = spl[0].zfill(2)
                else:
                    spl[1] = spl[1].zfill(2)
                time_ok = f'{spl[0]}:{spl[1]}'
        if len(time_ok) < 5:
            spl = [time_ok[0:2], time_ok[2:]]
            time_ok = f'{spl[0]}:{spl[1]}'

        new_series.append(time_ok)
    return pd.Series(new_series)

In [25]:
df2['time'] = time_formatter(df2.time)

In [26]:
time_lens = set()
for x in df2.time:
    time_lens.add(len(x))
print(time_lens)

{5}


In [27]:
for time in df2.time:
    cnt = 0
    for char in range(5):
        if time[char] == ':':
            cnt += 1
    if cnt > 1:
        print(f'PROBLEM FOUND: {x}')

### • Flight information columns (operator, flight_no, route, total_aboard, passengers_aboard, crew_aboard)

In [28]:
import requests

In [29]:
df2.operator.unique().shape

(2268,)

In [30]:
'?' in df2.operator.unique()

True

In [31]:
url = "https://aviation-reference-data.p.rapidapi.com/airline/search"

operator_dict = {}

for x in pd.Series(df2.operator.unique()).sample(20):
    if x != '?':
        querystring = {"name":x}
        headers = {
            "X-RapidAPI-Key": "ad8fe13e2fmsheba48590580e9bbp15cb61jsn64c6c2c5036e",
            "X-RapidAPI-Host": "aviation-reference-data.p.rapidapi.com"
        }
        response = requests.request("GET", url, headers=headers, params=querystring)
        print(f'-----------------------------\n{x}')
        print(response.text)
        operator_dict[x] = response.text

-----------------------------
Air East
[]
-----------------------------
General Air
[]
-----------------------------
Catalina Airlines
[]
-----------------------------
Finnair
[{"icaoCode":"FIN","iataCode":"AY","name":"FINNAIR","callSign":"FINNAIR","alpha3countryCode":"FIN"}]
-----------------------------
Lake Central Airlines
[]
-----------------------------
UTA
[{"icaoCode":"BTN","iataCode":"B3","name":"BHUTAN AIRLINES","callSign":"BHUTAN AIR","alpha3countryCode":"BTN"},{"icaoCode":"UTA","iataCode":"UT","name":"UTAIR AVIATION JSC","callSign":"UTAIR","alpha3countryCode":"RUS"},{"icaoCode":"RUC","iataCode":"5R","name":"RUTACA","callSign":"RUTACA","alpha3countryCode":"VEN"}]
-----------------------------
United Express  / Private
{"timestamp":"2022-11-08T02:10:06.868+00:00","status":400,"error":"Bad Request","message":"","path":"/airline/search"}
-----------------------------
Military -Royal Air Force
{"timestamp":"2022-11-08T02:10:07.375+00:00","status":400,"error":"Bad Request","messa

In [32]:
url = "https://aviation-reference-data.p.rapidapi.com/airline/search"

querystring = {"name":"Cabo Verde Airlines"}

headers = {
	"X-RapidAPI-Key": "ad8fe13e2fmsheba48590580e9bbp15cb61jsn64c6c2c5036e",
	"X-RapidAPI-Host": "aviation-reference-data.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers, params=querystring)

print(response.text)

[]


In [33]:
df2.loc[df2.all_aboard == '?']

,date,time,accident_location,operator,flight_no,route,ac_type,registration,cn_ln,all_aboard,passengers_aboard,crew_aboard,total_fatalities,passenger_fatalities,crew_fatalities,ground_fatalities,summary,year
26,1919-07-21,16:55,"Chicago, Illinois",Wingfoot Air Express Goodyear Tire,?,Shuttle,FD Type Dirigible,?,?,?,?,?,3,2,1,10,"The dirigible, cruising at 1,200 ft. caught fi...",1919
103,1925-09-07,00:00,"Toul, France",CIDNA,?,?,Spad 33,?,?,?,?,?,?,?,?,?,?,1925
347,1934-08-10,00:00,"Ningbo, China",China National Aviation Corporation,?,?,Sikorsky S-38B,?,?,?,?,?,?,?,?,?,?,1934
384,1935-08-13,00:00,"Hangow, China",China National Aviation Corporation,?,?,Sikorsky S-38B,NV40V,?,?,?,?,?,?,?,?,Destoryed in a storm.,1935
451,1936-12-26,00:00,"Nanking, China",China National Aviation Corporation,?,?,Douglas DC-2,NC14269,?,?,?,?,?,?,?,?,?,1936
652,1942-10-01,00:00,"Kunming, China",China National Aviation Corporation,?,?,Douglas C-47,69,?,?,?,?,?,?,?,?,Crashed while attempting to land after losing ...,1942
754,1944-11-09,00:00,"Seljord, Norway",Military - U.S. Army Air Corps,?,?,?,42-52196,?,?,?,?,?,?,?,?,?,1944
854,1946-03-18,00:00,"Between Chungking and Shanghai, China",China National Aviation Corporation,?,Chunking - Shanghai,?,139,?,?,?,?,?,?,?,?,Disappeared while en route. Plane never located.,1946
1231,1951-01-20,00:00,"Huilyo, Peru",Military - TAM,?,?,Douglas DC-3,?,?,?,?,?,16,?,?,?,?,1951
1362,1952-12-22,00:00,"Damascus, Syria",Syrian Air,?,?,Douglas DC-3,?,?,?,?,?,9,?,?,?,?,1952


### • Aircraft information columns (ac_type, registration, cn_ln)

### • Accident information columns (accident__location, total_fatalities, passenger_fatalities, crew_fatalities, ground_fatalities, summary)

In [34]:
# df_t.loc[df2.passenger_fatalities != (df2.crew_fatalities + df2.passenger_fatalities +)]
df2.loc[df2.passenger_fatalities == df2.crew_fatalities].passenger_fatalities.value_counts()

?     231
1      78
0      74
2      63
3      23
4      11
5       4
7       3
8       3
9       3
6       2
21      1
Name: passenger_fatalities, dtype: int64

In [36]:
df2.loc[df2.passenger_fatalities == '?'].ground_fatalities.value_counts()

0     212
?      16
53      1
37      1
36      1
5       1
1       1
4       1
3       1
Name: ground_fatalities, dtype: int64

In [37]:
df2.loc[(df2.passenger_fatalities == '?') & (df2.crew_fatalities != '?')]

,date,time,accident_location,operator,flight_no,route,ac_type,registration,cn_ln,all_aboard,passengers_aboard,crew_aboard,total_fatalities,passenger_fatalities,crew_fatalities,ground_fatalities,summary,year
628,1942-01-20,00:00,"Kupang, Indonesia",Military - Royal Australian Air Force,?,?,Lockheed Hudson,A16-79,414-1933,11,7,4,11,?,4,0,Shortly after taking off the left engine caugh...,1942
1306,1952-02-07,00:00,"Kaneko, Japan",Military - U.S. Air Force,?,?,Boeing B-29,?,?,13,?,13,13,?,13,5,The military plane crashed into several houses...,1952
1825,1960-04-18,00:00,"Stephenville, Newfoundland",Military - U.S. Air Force,?,Dover-Stephenville - Lajes - Mildenhall,Douglas C-124C Globemaster,52-1062,43971,9,?,9,9,?,9,0,Failed to gain altitude on take off and struck...,1960
4770,2010-07-05,17:40,"Tuzla, Romania",Military - Romainian Air Force,?,Tuzla - Tuzla,Antonov AN-2,53,1G194-53,14,10,4,12,?,3,0,"The military plane, on parachuting training fl...",2010


In [40]:
from geopy.geocoders import Nominatim, ArcGIS
from geopy.extra.rate_limiter import RateLimiter

geolocator = Nominatim(user_agent='acidminded')
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)
nom = ArcGIS()

In [41]:
loc_coors = {}
problematic_locations = []

In [43]:
'''
for x in df2.accident_location:
    coor = nom.geocode(query=x)
    try:
        loc_coors[x] = {'lat':coor.latitude, 'lon':coor.longitude}
        #print(f'\nQuery: {x}\nResult: {coor}\n • Lat: {coor.latitude}\n • Lon: {coor.longitude}\n--------------------------------')
    except AttributeError:
        problematic_locations.append(x)
        #print(f'Problem found: {x}')
    

print(problematic_locations)

# OUTPUT:

# ['Rossaugpt, Czechoslovakia', '?', '?', '?', '?', 'Batataevka, USSR', 'Near Hourghada, UAR', 'Near Adler, USSR', '?', 'Near Enisseysk, USSR', 'Southern Belarus, USSR', 'Near Kutayissi, USSR', 'Talourow Island, USSR', 'Nagoro-Karabak, USSR']
'''

['Rossaugpt, Czechoslovakia', '?', '?', '?', '?', 'Batataevka, USSR', 'Near Hourghada, UAR', 'Near Adler, USSR', '?', 'Near Enisseysk, USSR', 'Southern Belarus, USSR', 'Near Kutayissi, USSR', 'Talourow Island, USSR', 'Nagoro-Karabak, USSR']


In [55]:
updated_locations = {'Rossaugpt, Czechoslovakia': ['Rozvadov, Czech Republic'],
                     'Batataevka, USSR': ['Batayevka, Russia'],
                     'Near Hourghada, UAR': ['Hurghada, Egypt'],
                     'Near Adler, USSR': ['Adler, Russia'],
                     'Near Enisseysk, USSR': ['Yeniseysk , Russia'],
                     'Southern Belarus, USSR': ['Pinsk, Belarus'],
                     'Near Kutayissi, USSR': ['Kutaisi, Georgia'],
                     'Talourow Island, USSR': ['Taymyr Island, Russia'],
                     'Nagoro-Karabak, USSR': ['Nagorno-Karabakh,  Azerbaijan']
                     }

In [57]:
# Here we get the coordinates for the problematic locations using their updated names
'''
problematic_locations2 = []
for x in updated_locations:
    new_q = updated_locations[x][0]
    coor = nom.geocode(query=new_q)
    try:
        loc_coors[x] = {'lat':coor.latitude, 'lon':coor.longitude}
        #print(f'\nQuery: {x}\nResult: {coor}\n • Lat: {coor.latitude}\n • Lon: {coor.longitude}\n--------------------------------')
    except AttributeError:
        problematic_locations2.append(x)
        print(f'Problem found: {x}')
'''

In [58]:
# loc_coors['Near Adler, USSR']

{'lat': 43.43611000000004, 'lon': 39.93193000000008}

In [59]:
# loc_coors['Kaneko, Japan']

{'lat': 38.90868165000006, 'lon': 139.87539325000012}

In [61]:
# loc_coors['?'] = {'lat':0, 'lon':0}

In [62]:
# Here we save the queried data into a json file
'''
with open(r'.\coordinates.json', 'w') as coor_file:
    json.dump(loc_coors, coor_file)
'''

In [63]:
with open('coordinates.json') as json_file:
    loc_coors_r = json.load(json_file)

In [64]:
loc_coors_r['?']

{'lat': 0, 'lon': 0}

In [ ]:
url = "https://dark-sky.p.rapidapi.com/37.774929,-122.419418,2019-02-20"

headers = {
	"X-RapidAPI-Key": "ad8fe13e2fmsheba48590580e9bbp15cb61jsn64c6c2c5036e",
	"X-RapidAPI-Host": "dark-sky.p.rapidapi.com"
}

response = requests.request("GET", url, headers=headers)

print(response.text)